# IESB - CIA035 - Aula 05
# Random Forest x GBM x XGBoost x AdaBoost x CatBoost

Todos os métodos usam, basicamente, `árvore de decisão` como modelo de base e são chamados de `métodos ensemble`, que se caracterizam como métodos que procuram usar modelos mais simples/fracos de forma conjunta para melhorar o desempenho do modelo principal.

## Random Forest
Para obter melhor resultado, o algoritimo de RF vai criar diversas `árvores de decisão` (parâmetro `n_estimators`) e chegar ao resultado final com base no resultado de cada árvore criada. A idéia básica é separar o conjunto de dados diversas vezes e para cada sub-conjunto treinar um novo regressor/classificador.  Os diferentes regressores/classificadores irão produzir resultados diferentes, e o resultado final será determinado com base nessas regressões/classificações individuais.

## Gradient Boosting
GBM é um método de `boosting`, também construído em cima de regressores/classificadores fracos. A idéia é adicionar um regressor/classificador de cada vez, então o próximo regressor/classificador é treinado para melhorar o resultado atingido até o momento ('soma de resultados'). Ao contrário do RF, que treina cada regressor/classificador de forma independente, no GBM eles são treinados em conjunto, um ligado ao outro.

## XGBoost
XGB é uma implementação específica do GBM, dita melhor e mais rápida que a implementação padrão do scikit-learn. Tanto o GBM quanto o XGB precisam de maior trabalho de interpretação dos dados e `tunning` do modelo.

## AdaBoost
Inicialmente o modelo é treinado nos dados originais e então cópias desse modelo são treinadas no mesmo conjunto dados mas com ajuste de peso em cima das ocorrências classificadas incorretamente, de forma que os modelos subsequentes tenham foco nos casos mais difícies de serem previstos.

## CatBoost
É um modelo mais otimizado de boosting usando árvores de decisão que tem a vantagem de trabalhar com `dados categóricos`, eliminando a necessidade de pré-processamento, além de ser otimizado para execução em `GPU`.

In [ ]:
# Importando as bibliotecas
import numpy as np
import pandas as pd

# Verificando os arquivos
import os
print(os.listdir("../input"))

In [ ]:
# Carregando os dados
df = pd.read_csv('../input/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.head().T

In [ ]:
# Verificando os tipos e os valores nulos
df.info()

In [ ]:
# Para corrigir o TotalCharges vamos trocar o espaço em branco
# pelo valor ZERO e forçar a conversão
df['TotalCharges'] = df['TotalCharges'].str.replace(' ', '0').astype(float)

In [ ]:
# Guardando as colunas categóricas para execução do CatBoost
df2 = df.copy()

In [ ]:
# Transformar texto em número
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

In [ ]:
df.info()

In [ ]:
# Dividindo o DataFrame
from sklearn.model_selection import train_test_split

# Treino e teste
train, test = train_test_split(df, test_size=0.15, random_state=42)

# Veificando o tanho dos DataFrames
train.shape, test.shape

In [ ]:
# Selecionado as features
feats = [c for c in df.columns if c not in ['customerID', 'Churn']]

In [ ]:
# Trabalhando com RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=200, min_samples_split=5, max_depth=4, random_state=42)
rf.fit(train[feats], train['Churn'])
accuracy_score(test['Churn'], rf.predict(test[feats]))

In [ ]:
df['Churn'].value_counts(normalize=True)

In [ ]:
# Trabalhando com GBM
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(train[feats], train['Churn'])
accuracy_score(test['Churn'], gbm.predict(test[feats]))

In [ ]:
# Trabalhando com XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=200, learning_rate=0.01, random_state=42, use_label_encoder=False)
xgb.fit(train[feats], train['Churn'])
accuracy_score(test['Churn'], xgb.predict(test[feats]))

In [ ]:
# Trabalhando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(n_estimators=200, learning_rate=0.5, random_state=42)
abc.fit(train[feats], train['Churn'])
accuracy_score(test['Churn'], abc.predict(test[feats]))

In [ ]:
# Trabalhando com CatBoost
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(random_state=42)
cbc.fit(train[feats], train['Churn'])
accuracy_score(test['Churn'], cbc.predict(test[feats]))

In [ ]:
# Criando uma lista de colunas categoricas
cat_cols = [c for c in df2.columns if ((df2[c].dtype == 'object') and (c not in ['customerID', 'Churn']))]

cat_cols

In [ ]:
# Verificando os tipos
df2.info()

In [ ]:
# Usando o CatBoost com o dataframe sem processamento

# Treino 2 e teste 2
train2, test2 = train_test_split(df2, test_size=0.15, random_state=42)

cbc2 = CatBoostClassifier(random_state=42, cat_features=cat_cols)
cbc2.fit(train2[feats], train2['Churn'])
accuracy_score(test2['Churn'], cbc2.predict(test2[feats]))

In [ ]:
# Feature Importance com RF
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Feature Importance com GBM
pd.Series(gbm.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Feature Importance com XGB
pd.Series(xgb.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Feature Importance com AdaBoost
pd.Series(abc.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Feature Importance com CatBoost
pd.Series(cbc.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Feature Importance com CatBoost (usando dados categóricos)
pd.Series(cbc2.feature_importances_, index=feats).sort_values().plot.barh()